In [48]:
import pandas as pd

df = pd.read_csv('UL-Standards-List.csv')
# drop rows with no description
df = df.dropna(subset=['Description'])
len(df)
# df.head()


2123

In [49]:
# Description to Title Case since there are some descriptions that are all caps
df['Description'] = df['Description'].str.title()





In [50]:

df.iloc[144]['Description']

'Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method) / Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method)'

In [51]:
# Removing duplicates in the description separated by ' / '
def remove_duplicates(string):
    words = string.split(' / ')
    result = []
    for word in words:
        if word not in result:
            result.append(word)
    return ' / '.join(result)  # nothing to join if there are no duplicates

df['Description'] = df['Description'].apply(remove_duplicates)
df.iloc[570]['Description']

'Standard For Category 3 Film-Forming Fluoroprotein (Fffp) Foam Liquid Concentrates'

In [52]:
# Look for non-English descriptions
def isEnglish(s):
    return s.isascii()

df['English?'] = df['Description'].apply(isEnglish)


In [53]:
df['Description'] = df['Description'].str.replace(' For ', ' for ')
df['Description'] = df['Description'].str.replace(' Of ', ' of ')

In [54]:
df['Category'] = ''
df['Topic'] = ''

In [55]:
# Remove non-English descriptions
for idx, row in df.iterrows():
    if row['English?'] == False:
        split_description = row['Description'].split(' / ', maxsplit=1)
        if len(split_description) > 1:
            df.at[idx, 'Topic'] = split_description[0]
        else:
            df.at[idx, 'Topic'] = row['Description']
    else:
        df.at[idx, 'Topic'] = row['Description']



In [56]:
df['Topic'][1505]

'Ieee/Ul Wireless Diabetes Device Security: Information Security Requirements for Connected Diabetes Solutions'

In [57]:
import re
def find_words_in_description(string, words):
    start_index = 0
    found_word = ''
    for word in words:
        i = string.find(word, start_index)
        if i != -1:
            if i > 40:
                continue
            else:
                start_index = i + len(word)

            found_word = found_word + ' ' + word
        else:
            start_index = start_index
    return (start_index, found_word)

words = ['Standard', 'Method', 'Test', 'Outline', 'Guide', 'Guidance', 'Criteria', 'Procedure', 'Rule', 'Requirement','Enclosure']


for idx, row in df.iterrows():
    i = find_words_in_description(row['Topic'], words)[0]
    final_word = find_words_in_description(row['Topic'], words)[1]
    print(i, final_word)
    if final_word != '':
        df.at[idx, 'Category'] = final_word
        split_topic_desc = re.split(' for | of ', row['Topic'][i:], maxsplit=1)
        # join the word in description before the split if not in the list of words

        print(split_topic_desc)
        if len(split_topic_desc) > 1:
            lost_words = [word for word in row['Topic'][:i] if word not in words]
            new_string = ''.join(lost_words)
            df.at[idx, 'Category'] = new_string
            df.at[idx, 'Topic'] = split_topic_desc[1]

    else:
        pass

8  Standard
[' for Flexible Metal Conduit']
8  Standard
[' for Armored Cable']
8  Standard
[' for Surface Metal Raceways And Fittings']
0 
8  Standard
[' for Strut-Type Channel Raceways And Fittings']
8  Standard
[' for Surface Raceways And Fittings for Use With Data, Signal, And Control Circuits']
0 
8  Standard
[' for Electrical Rigid Metal Conduit - Aluminum And Stainless Steel']
7  Outline
[' of Investigation for Pump Tank Extinguishers']
0 
22  Standard Test
['s of Window Assemblies']
0 
5  Guide
[' for Evaluating Doors And Frames for Listing And Labelling']
8  Standard
[' for Tin-Clad Fire Doors']
22  Standard Test
['s of Door Assemblies']
0 
40  Standard Test
['s of Door Assemblies']
22  Standard Test
['s of Fire-Protective Curtain Assemblies']
8  Standard
[' for Power-Limited Circuit Cables']
5  Guide
[' for Hardware for Fire Doors And Emergency Exits']
8  Standard
[' for Sliding Hardware for Standard, Horizontally Mounted Tin-Clad Fire Doors']
8  Standard
[' for Swinging Hardw

In [58]:
df['Category']

0           Standard
1           Standard
2           Standard
3                   
4           Standard
            ...     
2118     Requirement
2119                
2120                
2121                
2122                
Name: Category, Length: 2123, dtype: object

In [45]:
df.to_csv('Results_3.csv', index=False)

In [4]:
# from top2vec import Top2Vec
# model = Top2Vec(documents=df['Description'].tolist())



KeyboardInterrupt

